In [ ]:
# State the problem number
problem_number = '0'

# and the user id (given during registration)
user_id = '68'

# name of this submission (no space or special character)

name = "3gram_Baseline"

train_file = 'Data/0.spice.train.txt'
prefix_file = 'Data/0.spice.public.test.txt'

In [18]:
from numpy import *
from decimal import *
from sys import *
def number(arg):
    return Decimal(arg)
def threegramdict(sett):
    DPdict = dict()
    total = 0
    #sett is a List of Lists(Holds All Sequences where each Sequence is a list of Observations)
    for sequence in sett:
        ngramseq = [-1,-1] + sequence + [-2]#Appends -1,1 before sequence and -2 after sequence
        for start in range(len(ngramseq)-2):
            total
            end = start + 2#Because first 2 elements -1,-1 are Dummy
            if tuple(ngramseq[start:end]) in DPdict:#If this Prefix pair has already been seen
                table = DPdict[tuple(ngramseq[start:end])]
                if ngramseq[end] in table:
                    table[ngramseq[end]] = table[ngramseq[end]] + 1
                else:
                    table[ngramseq[end]] = 1
                table[-1] = table[-1] + 1#Increase Counter for Key -1:It eventually stores how many times Prefix Pair was seen
            else:
                table = dict()
                table[ngramseq[end]] = 1#New Dictionary Object With only 1 entry for 3rd Element for this never seen Prefix-Pair
                table[-1] = 1#stores how many times Prefix Pair was seen: It's now seen 1 time :-)
                DPdict[tuple(ngramseq[start:end])] = table#Add this newly created dictionary into the global Prefix DP
    return DPdict

def threegramrank(prefix, alphabet, DPdict):
    probs=[]
    # Compute the probability for prefix to be a whole sequence
    prob = number('1.0')
    ngramseq = [-1,-1] + prefix + [-2]#Appends -1,1 before sequence and -2 after sequence
    for start in range(len(ngramseq)-2):
        end = start + 2#Because initial 2 values are -1,-1
        #Below Logic: P(C|A,B) using Principal of Maximum Likelihood using Empirical Method
        if tuple(ngramseq[start:end]) in DPdict and ngramseq[end] in DPdict[tuple(ngramseq[start:end])]:
            prob = prob * (number(DPdict[tuple(ngramseq[start:end])][ngramseq[end]]) / number(DPdict[tuple(ngramseq[start:end])][-1]))
        else:
            # Subsequence not in the dictionnary
            prob = number(0)
    #print("Probability of the Whole Sequence ",prob," ", ngramseq)
    probs.append((-1,prob))
    #Compute Probability of each possible next sequence (i.e. by appending all possible Outcomes at next position)
    for x in range(alphabet):
        prob = number('1.0')
        ngramseq = [-1,-1] + prefix + [x]
        for start in range(len(ngramseq)-2):
            end = start + 2
            if tuple(ngramseq[start:end]) in DPdict and ngramseq[end] in DPdict[tuple(ngramseq[start:end])]:
                prob = prob * (number(DPdict[tuple(ngramseq[start:end])][ngramseq[end]]) / number(DPdict[tuple(ngramseq[start:end])][-1]))
            else:
                # Subsequence not in the dictionnary
                prob=number(0)
        probs.append((x,prob))
        #print(x,prob,ngramseq)
    print(probs)
    probs=sorted(probs, key=lambda x: -x[1])
    return [x[0] for x in probs]

In [19]:
def readset(f):
    sett = []
    line = f.readline()
    l = line.split(" ")
    num_strings = int(l[0])#Total Number of Sequences
    alphabet_size = int(l[1])#Total Number of Distinct Observations
    for n in range(num_strings):
        line = f.readline()#Reading Sequences 1 by 1
        l = line.split(" ")
        sett = sett + [[int(i) for i in l[1:len(l)]]]#sett is a 2-Dimensional Array(num_strings*Number_Of_Observations_in_String)
    return alphabet_size, sett
def get_first_prefix(test_file):
    """ get the only prefix in test_file
        This function is called for the public test file(Which only has 1 line)
    """
    f = open(test_file)
    prefix = f.readline()
    f.close()
    return prefix

def list_to_string(l):
    s=str(l[0])
    for x in l[1:]:
        s+= " " + str(x)
    return(s)

def formatString(string_in):
    """ Replace white spaces by %20 """
    return string_in.strip().replace(" ", "%20")


In [20]:
print("Get training sample")
alphabet, train = readset(open(train_file,"r"))
print ("Start Learning")
dict1=threegramdict(train)
print ("Learning Ended")
print(len(dict1),dict1)

Get training sample
Start Learning
Learning Ended
18 {(0, 1): {0: 934, 1: 1342, 2: 1476, 3: 3052, -2: 518, -1: 7322}, (1, 2): {0: 1543, 1: 1278, 2: 453, 3: 1254, -1: 5060, -2: 532}, (3, 2): {0: 1211, 1: 350, 2: 158, 3: 402, -1: 2739, -2: 618}, (1, 3): {0: 3196, 1: 1372, 2: 538, 3: 4874, -1: 12377, -2: 2397}, (3, 3): {0: 5775, 1: 3758, 2: 1226, 3: 2789, -2: 3923, -1: 17471}, (2, 0): {0: 495, 1: 726, 2: 541, 3: 2651, -1: 4827, -2: 414}, (3, 0): {0: 2362, 1: 4471, 2: 4379, 3: 8334, -1: 22594, -2: 3048}, (2, 2): {0: 549, 1: 101, 2: 90, 3: 247, -2: 87, -1: 1074}, (3, 1): {0: 1420, 1: 1803, 2: 2348, 3: 6159, -1: 12262, -2: 532}, (1, 1): {0: 524, 1: 2613, 2: 836, 3: 2287, -2: 413, -1: 6673}, (2, 1): {0: 331, 1: 915, 2: 400, 3: 879, -2: 230, -1: 2755}, (0, 2): {0: 1524, 1: 1026, 2: 373, 3: 1913, -1: 5776, -2: 940}, (-1, 3): {0: 7635, 1: 3302, 2: 515, 3: 7373, -2: 1175, -1: 20000}, (0, 0): {0: 668, 1: 1030, 2: 350, 3: 941, -1: 3948, -2: 959}, (2, 3): {0: 1305, 1: 819, 2: 270, 3: 313, -2: 1109, 

In [21]:
# get the test first prefix: the only element of the test set
first_prefix = get_first_prefix(prefix_file)
prefix_number=1
# get the next symbol ranking on the first prefix
p=first_prefix.split()
prefix=[int(i) for i in p[1:len(p)]]#prefix holds the sequence of values in the public test file(Note:It has only 1 Seq)
ranking = threegramrank(prefix, alphabet, dict1)#alphabet:Maximum possible number of distinct observations
print(ranking)
ranking_string=list_to_string(ranking[:5])
print("Prefix number: " + str(prefix_number) + " Ranking: " + ranking_string + " Prefix: " + first_prefix)

[(-1, Decimal('0.0003855316610227944742408134652')), (0, Decimal('0.0005675364115234866400052760035')), (1, Decimal('0.0003693163349792662845263770079')), (2, Decimal('0.0001204847862385791550902975550')), (3, Decimal('0.0002740881474872734612943229046'))]
[0, -1, 1, 3, 2]
Prefix number: 1 Ranking: 0 -1 1 3 2 Prefix: 7 3 0 3 0 1 3 3


In [23]:
# transform the first prefix to follow submission format
first_prefix = formatString(first_prefix)

# transform the ranking to follow submission format
ranking_string=formatString(ranking_string)

# create the url to submit the ranking
name=name+"_Ver4"
url_base = 'http://spice.lif.univ-mrs.fr/submit.php?user=' + user_id +\
    '&problem=' + problem_number + '&submission=' + name + '&'
url = url_base + 'prefix=' + first_prefix + '&prefix_number=1' + '&ranking=' +\
    ranking_string

# Get the website answer for the first prefix with this ranking using this
# submission name
try:
    # Python 2.7
    import urllib2 as ur
    orl2 = True
except:
    #Python 3.4
    import urllib.request as ur
    orl2 = False

response = ur.urlopen(url)
print("URL ",url)
content = response.read()
print("Response from SPiCe ",content)#Content is a new Sequence returned from the SPiCe server: We will need to predict for this seq
if not orl2:
    # Needed for python 3.4...
    content= content.decode('utf-8')

list_element = content.split()
head = str(list_element[0])

prefix_number = 2

URL  http://spice.lif.univ-mrs.fr/submit.php?user=68&problem=0&submission=3gram_Baseline_Ver3_Ver4_Ver4&prefix=7%203%200%203%200%201%203%203&prefix_number=1&ranking=0%20-1%201%203%202
Response from SPiCe  b'2 3 3 \n'


In [24]:
print(head,"Content :", content[:-1])

2 Content : 2 3 3 


In [25]:
while(head != '[Error]' and head != '[Success]'):
    prefix = content[:-1]#Fetch the Sequence returned from Spice Server and exclude the last '\n'
    # Get the ranking
    p=prefix.split()
    prefix_list= list()
    prefix_list=[int(i) for i in p[1:len(p)]]
    ranking = threegramrank(prefix_list, alphabet, dict1)#Use this new Sequence prefix_list to predict next sequence
    ranking_string=list_to_string(ranking[:5])#Here At least alphabet should be 4: Else may get Runtime error
    
    print("Prefix number: " + str(prefix_number) + " Ranking: " + ranking_string + " Prefix: " + prefix)
    
    # Format the ranking
    ranking_string = formatString(ranking_string)
    
    # create prefix with submission needed format
    prefix=formatString(prefix)
    
    # Create the url with your ranking to get the next prefix
    url = url_base + 'prefix=' + prefix + '&prefix_number=' +\
        str(prefix_number) + '&ranking=' + ranking_string
    
    # Get the answer of the submission on current prefix
    response = ur.urlopen(url)
    content = response.read()
    if not orl2:
        # Needed for Python 3.4...
        content= content.decode('utf-8')
    
    list_element = content.split()
    # modify head in case it is finished or an erro occured
    head = str(list_element[0])
    # change prefix number
    prefix_number += 1

# Post-treatment
# The score is the last element of content (in case of a public test set)
print(content)

list_element = content.split()
score = (list_element[-1])
print(score)

[(-1, Decimal('0.08277797206799839734417033943')), (0, Decimal('0.1218564335184019231869955927')), (1, Decimal('0.07929635968175834239597046534')), (2, Decimal('0.02586943506382004464541239769')), (3, Decimal('0.05884979966802129242745120487'))]
Prefix number: 2 Ranking: 0 -1 1 3 2 Prefix: 2 3 3 
[(-1, Decimal('1.676144804948302071906189434E-9')), (0, Decimal('3.022237930157749295676411066E-9')), (1, Decimal('4.342444649113168688220282279E-9')), (2, Decimal('4.776041953868134861261651747E-9')), (3, Decimal('9.875663985911617612852683693E-9'))]
Prefix number: 3 Ranking: 3 2 1 0 -1 Prefix: 12 3 1 0 0 1 0 1 3 0 2 0 1 
[(-1, Decimal('0.0001633033674901868009774166539')), (0, Decimal('0.0002177378233202490679698888718')), (1, Decimal('0.00009347193166313570752649797631')), (2, Decimal('0.00003665298778044242758692121811')), (3, Decimal('0.0003320569933863873458339293997'))]
Prefix number: 4 Ranking: 3 0 -1 1 2 Prefix: 7 3 3 0 2 3 1 3 
[(-1, Decimal('0')), (0, Decimal('0')), (1, Decimal('0')

URLError: <urlopen error [Errno -2] Name or service not known>